In [13]:
import pandas as pd
import ast
from tqdm import tqdm

tqdm.pandas()
df = pd.read_csv('infomax_docs.csv')
df['tagged'] = df['tagged'].progress_apply(ast.literal_eval)

100%|██████████| 79888/79888 [03:08<00:00, 423.45it/s]


In [ ]:
# tagged 컬럼 삭제 버전
# from collections import Counter
# from tqdm import tqdm

# ALLOWED_TAGS = {'NNG', 'VA', 'VAX', 'MAG', 'negations'}

# def generate_ngrams(token_list, max_n=5, allowed_tags=ALLOWED_TAGS):
#     words = [token[0] for token in token_list if token[1] in allowed_tags]
#     ngram_counter = Counter()
#     for n in range(1, max_n + 1):
#         ngrams = [tuple(words[i:i+n]) for i in range(len(words) - n + 1)]
#         ngram_counter.update(ngrams)
#     return ngram_counter

# def sorted_ngrams(token_list, max_n=5):
#     counter = generate_ngrams(token_list, max_n)
#     sorted_ng = sorted(counter.items(), key=lambda x: x[1], reverse=True)
#     return str(sorted_ng)

# tqdm.pandas()
# # 각 행별로 n-그램 결과를 'ngram' 컬럼에 추가
# df['ngram'] = df['tagged'].progress_apply(lambda x: sorted_ngrams(x, max_n=5))
# df[['date', 'ngram']].to_csv('ngram_edaily_results.csv', index=False)

100%|██████████| 5367/5367 [00:16<00:00, 320.20it/s]


In [14]:
# 날짜별 그룹화, 빈도수 필터
from collections import Counter
from tqdm import tqdm
import csv

ALLOWED_TAGS = {'NNG', 'VA', 'VAX', 'MAG', 'negations'}

def generate_ngrams(token_list, max_n=5, allowed_tags=ALLOWED_TAGS):
    words = [token[0] for token in token_list if token[1] in allowed_tags]
    ngram_counter = Counter()

    for n in range(1, max_n + 1):
        ngrams = [tuple(words[i:i+n]) for i in range(len(words) - n + 1)]
        ngram_counter.update(ngrams)
    return ngram_counter

def sorted_ngrams(token_list, max_n=5):
    counter = generate_ngrams(token_list, max_n)
    sorted_ng = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    return str(sorted_ng)

tqdm.pandas()
df['ngram_counter'] = df['tagged'].progress_apply(lambda x: generate_ngrams(x, max_n=5))

with open('ngram_infomax_results_filtered.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['date', 'filtered_ngram'])
    
    # 고유한 날짜마다 처리
    for date in df['date'].unique():
        subset = df[df['date'] == date]
        total_counter = Counter()
        for counter in subset['ngram_counter']:
            total_counter.update(counter)
        # 빈도수가 15회 이상인 토큰만 필터링
        filtered_counter = [(token, count) for token, count in total_counter.items() if count >= 15]
        writer.writerow([date, str(filtered_counter)])

100%|██████████| 79888/79888 [00:32<00:00, 2431.19it/s]


In [3]:
pd.read_csv('ngram_bond_results.csv')['ngram'][0]

"[(('금리',), 13), (('투자',), 10), (('회사',), 10), (('크레딧',), 7), (('제공',), 6), (('증권',), 6), (('투자', '증권'), 6), (('유동성',), 5), (('유효',), 5), (('회사채',), 5), (('축',), 5), (('상기',), 5), (('상기', '회사'), 5), (('추가',), 4), (('금통위',), 4), (('사채',), 4), (('관련',), 4), (('은행',), 4), (('제시',), 4), (('판단',), 4), (('의견',), 4), (('국내',), 3), (('기대',), 3), (('우려',), 3), (('약세',), 3), (('거래',), 3), (('예금',), 3), (('대비',), 3), (('인하',), 3), (('공사',), 3), (('주식',), 3), (('조사',), 3), (('분석',), 3), (('국내', '크레딧'), 3), (('금리', '축'), 3), (('조사', '분석'), 3), (('중심',), 2), (('이슈',), 2), (('일단락',), 2), (('시장',), 2), (('부각',), 2), (('하락',), 2), (('국채',), 2), (('속',), 2), (('신용스프레드',), 2), (('유지',), 2), (('스프레드',), 2), (('폭',), 2), (('확대',), 2), (('반',), 2), (('발생',), 2), (('채권',), 2), (('강세',), 2), (('재무비율',), 2), (('내용',), 2), (('사채권자',), 2), (('집회',), 2), (('소집',), 2), (('언론',), 2), (('보증',), 2), (('그룹',), 2), (('현금',), 2), (('포함',), 2), (('가용',), 2), (('자금',), 2), (('규모',), 2), (('보유',), 2), (('자산',), 2), (('압력',